In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
init_load_flag = int(dbutils.widgets.get("init_load_flag"))

### **Data Reading**

In [0]:
df = spark.sql("select * from databricks_cata.silver.customers_silver")

In [0]:
df.display()

### **Removing Duplicates**

In [0]:
df = df.dropDuplicates(subset=['customer_id'])

### **Dividing New vs Old Records**

In [0]:
if init_load_flag ==0:

    df_old = spark.sql("select DimCustomerKey, customer_id, create_date, update_date from databricks_cata.gold.DimCustomers")

else:
    
    df_old = spark.sql('''select 0 DimCustomerKey, 0 customer_id, 0 create_date, 0 update_date from databricks_cata.silver.customers_silver where 1=0''')

In [0]:
df_old.display()

### **Surrogate Key - All the Values**

In [0]:
df = df.orderBy("customer_id").withColumn("DimCustomerKey", monotonically_increasing_id()+lit(1))

### **Renaming columns of df_old**

In [0]:
df_old = df_old.withColumnRenamed("DimCustomerKey", "old_DimCustomerKey")\
    .withColumnRenamed("customer_id", "old_customer_id")\
    .withColumnRenamed("create_date", "old_create_date")\
    .withColumnRenamed("update_date", "old_update_date")

### **Applying Join with Old Records**

In [0]:
df_join = df.join(df_old, df["customer_id"] == df_old["old_customer_id"], 'left')


In [0]:
df_join.display()

In [0]:
df.display()

### **Separating New vs Old Records**

In [0]:
 df_new = df_join.filter(df_join['old_DimCustomerKey'].isNull())

In [0]:
df_old = df_join.filter(df_join['old_DimCustomerKey'].isNotNull())

### **Preparing df_old**

In [0]:
# Dropping all the columns which are not required

df_old = df_old.drop('old_DimCustomerKey', 'old_customer_id','old_update_date')

# Renaming "old_create_date" to "create_date"

df_old = df_old.withColumnRenamed("old_create_date", "create_date")
df_old = df_old.withColumn("create_date",to_timestamp(col("create_date")))

# Recreating "update_date" column with current timestamp

df_old = df_old.withColumn("update_date", current_timestamp())

In [0]:
df_old.display()

### **Preparing Df_new**

In [0]:
df_new.display()

In [0]:
# Dropping all the columns which are not required

df_new = df_new.drop('old_DimCustomerKey', 'old_customer_id','old_update_date', 'old_create_date')

# Recreating "update_date, create_date" column with current timestamp

df_new = df_new.withColumn("update_date", current_timestamp())
df_new = df_new.withColumn("create_date", current_timestamp())

In [0]:
df_new.display()

### **Surrogate Key from 1**

In [0]:
df_new = df_new.withColumn("DimCustomerKey",monotonically_increasing_id()+lit(1))
df_new.display()

### **Adding Max Surrogate Key**

In [0]:
if init_load_flag == 1:
    max_surrogate_key = 0
else:
    df_maxsur = spark.sql("select max(DimCustomerKey) as max_surrogate_key from databricks_cata.gold.DimCustomers")

    # Converting df_maxsur to max_surrogate_key variable
    max_surrogate_key = df_maxsur.collect()[0]['max_surrogate_key']

In [0]:
df_new = df_new.withColumn("DimCustomerKey",lit(max_surrogate_key)+col("DimCustomerKey"))

In [0]:
df_new.display()

### **Union of df_old and df_new**

In [0]:
df_final = df_new.unionByName(df_old)

In [0]:
df_old.display()
df_final.display()

### **SCD type - 1**

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("databricks_cata.gold.DimCustomers"):
   dlt_obj = DeltaTable.forPath(spark, "abfss://gold@databricksete80.dfs.core.windows.net/DimCustomers")
   dlt_obj.alias("trg").merge(df_final.alias("src"),
    "trg.DimCustomerKey = src.DimCustomerKey")\
    .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
            .execute()
else:
    df_final.write.mode("overwrite")\
    .format("delta")\
        .option("path","abfss://gold@databricksete80.dfs.core.windows.net/DimCustomers")\
            .saveAsTable("databricks_cata.gold.DimCustomers")

In [0]:
%sql
select * from databricks_cata.gold.dimcustomers